# Big Data – Exercises - Solutions

# Fall 2022 -  Week 11 - RumbleDB



# 1. Install RumbleDB

We recommend you install RumbleDB locally. 

## Option 1: locally
To install the latest version of RumbleDB locally, you can follow the steps given on the official rumbledb website: 
https://rumble.readthedocs.io/en/latest/Getting%20started/

Please follow very carefully the instructions of [Method 1](https://rumble.readthedocs.io/en/latest/Getting%20started/#method-1-with-the-large-standalone-rumbledb-jar-experimental), in particular, download the big jar and execute it with the normal `java -jar` instruction, with Java 8 or 11. You can put the jar file in the working directory of this week. 

To launch a JSONiq shell with:

`java -jar rumbledb-1.20.0-standalone.jar repl`

This gives access to the RumbleDB shell where you can try commands. Note that to run each command, you should press 'Enter' two times, and you can quit with 'exit'. 

You need to make sure the Java version is correct and the JAVA_HOME environment variable is correctly set. 

E.g., on MacOS, (1) you can set `export JAVA_HOME=$(/usr/libexec/java_home -v 11)`; or (2) export `JAVA_HOME` to `/Library/Java/JavaVirtualMachines/jdk-11.0.1.jdk/Contents/Home/` or similar. On Linux/Windows OS there are similar methods to set `JAVA_HOME`. 


## Option 2: Github codespaces
If the local installation does not work for you, please use this option. 
See this [tutorial](https://cloud.inf.ethz.ch/s/KHLnq9HsQHfLHHQ/download) for running RumbleDB on Github codespaces. 


# 2. Setup RumbleDB in Jupyter Notebook



### Install Jupyter Notebook



In order to execute the queries in this notebook, you need to [install](https://jupyter.org/install) jupyter notebook on your **own machine**, and then download this notebook and [run](https://jupyter.readthedocs.io/en/latest/running.html#running) it locally.

From now we want to run the code on this notebook instead of the shell, hence you can close the shell (Ctrl+C or type 'exit') and execute your own Rumble server by running:

```
java -jar rumbledb-1.20.0-standalone.jar --server yes --port 9090
```

The mount allows RumbleDB to directly read the files from your local disk, in this case your exercise folder of this week. 

It is recommended to use the port 9090 which normally does not get occupied by other applications.  

To verify that the RumbleDB is up and running, you can go to `http://localhost:9090/public.html` in a web browser and you can type queries there.  
<img src="https://cloud.inf.ethz.ch/s/i9HGoCcDQxxBNKH/download" width=700>


To get started, you first need to execute the cell below to activate the RumbleDB magic (you do not need to understand what it does, this is just initialization Python code).

In [ ]:
!pip install rumbledb
%load_ext rumbledb
%env RUMBLEDB_SERVER=http://localhost:9090/jsoniq


Now we are all set! You can now start reading and executing the JSONiq queries as you go, and you can even edit them!

# 3. RumbleDB Sandbox
Note that for convenience, we reuse [the official Rumble Sandbox tutorial](https://github.com/RumbleDB/rumble/blob/master/RumbleSandbox.ipynb) in this section with minimal adaptation. 

## JSON

As explained on the [official JSON Web site](http://www.json.org/), JSON is a lightweight data-interchange format designed for humans as well as for computers. It supports as values:
- objects (string-to-value maps)
- arrays (ordered sequences of values)
- strings
- numbers
- booleans (true, false)
- null

JSONiq provides declarative querying and updating capabilities on JSON data. You can refer to [the documentation of JSONiq](https://www.jsoniq.org/docs/JSONiq/webhelp/index.html) for more detail.

## Elevator Pitch

JSONiq is based on XQuery, which is a W3C standard (like XML and HTML). XQuery is a very powerful declarative language that originally manipulates XML data, but it turns out that it is also a very good fit for manipulating JSON natively.
JSONiq, since it extends XQuery, is a very powerful general-purpose declarative programming language. Our experience is that, for the same task, you will probably write about 80% less code compared to imperative languages like JavaScript, Python or Ruby. Additionally, you get the benefits of strong type checking without actually having to write type declarations.
Here is an appetizer before we start the tutorial from scratch.


In [ ]:
%%jsoniq

let $stores :=
[
  { "store number" : 1, "state" : "MA" },
  { "store number" : 2, "state" : "MA" },
  { "store number" : 3, "state" : "CA" },
  { "store number" : 4, "state" : "CA" }
]
let $sales := [
   { "product" : "broiler", "store number" : 1, "quantity" : 20  },
   { "product" : "toaster", "store number" : 2, "quantity" : 100 },
   { "product" : "toaster", "store number" : 2, "quantity" : 50 },
   { "product" : "toaster", "store number" : 3, "quantity" : 50 },
   { "product" : "blender", "store number" : 3, "quantity" : 100 },
   { "product" : "blender", "store number" : 3, "quantity" : 150 },
   { "product" : "socks", "store number" : 1, "quantity" : 500 },
   { "product" : "socks", "store number" : 2, "quantity" : 10 },
   { "product" : "shirt", "store number" : 3, "quantity" : 10 }
]
let $join :=
  for $store in $stores[], $sale in $sales[]
  where $store."store number" = $sale."store number"
  return {
    "nb" : $store."store number",
    "state" : $store.state,
    "sold" : $sale.product
  }
return [$join]



## All JSON values are JSONiq, too

The first thing you need to know is that a well-formed JSON document is a JSONiq expression as well.
This means that you can copy-and-paste any JSON document into a query. The following are JSONiq queries that are "idempotent" (they just output themselves):

In [ ]:
%%jsoniq
{ "pi" : 3.14, "sq2" : 1.4 }

In [ ]:
%%jsoniq
[ 2, 3, 5, 7, 11, 13 ]

In [ ]:
%%jsoniq
{
      "operations" : [
        { "binary" : [ "and", "or"] },
        { "unary" : ["not"] }
      ],
      "bits" : [
        0, 1
      ]
    }

In [ ]:
%%jsoniq
[ { "Question" : "Ultimate" }, ["Life", "the universe", "and everything"] ]

This works with objects, arrays (even nested), strings, numbers, booleans, null.

It also works the other way round: if your query outputs an object, you can use it as a JSON document.
JSONiq is a declarative language. This means that you only need to say what you want - the compiler will take care of the how. 

In the above queries, you are basically saying: I want to output this JSON content, and here it is.

In fact JSONiq makes JSON "dynamic": try to replace numbers with arithmetic formulas, keys with concatenations of strings, etc and see how the resulting JSON object is automatically created.

In [ ]:
%%jsoniq
{
    "foo" : 2 + 2,
    "foo" || "bar" : if(2 gt 1) then true else false
}

## Navigating an existing JSON dataset

Next, let us look at an existing dataset on the Web. We picked a [GitHub archive file](https://gharchive.org)
that we stored for convenience at this location: https://www.rumbledb.org/samples/git-archive.json. 


The data schema is provided [here](https://github.com/igrigorik/gharchive.org/blob/master/bigquery/schema.js), which only includes common activity fields as per [https://gharchive.org](https://gharchive.org). 


Accessing a JSON dataset can be done in two ways depending on the exact format:

- If this is a file that contains a single JSON object spread over multiple lines, use json-doc(URL).
- If this is a file that contains one JSON object per line (JSON Lines), use json-file(URL).

The GitHub archive dataset is in the JSON Lines format, so we open it with json-file.

`git-archive-small.json` is the first 500 lines of `git-archive.json`. 



In [ ]:
%%jsoniq
json-file("http://www.rumbledb.org/samples/git-archive-small.json")

To view a JSON document in a formatted fashion, you can use oXygen or an online tool like [this](https://jsonformatter.org/json-viewer). A JSON document viewed as a tree can help you better understand the data structure. See the first JSON document in `git-archive-small.json` viewed as a tree. 
<img src="https://cloud.inf.ethz.ch/s/qcqWzck6Rdzmnim/download" width=900>

The previous query output 500 JSON objects. To look closer, let us start looking at just the first object with a number predicate.

In [ ]:
%%jsoniq
json-file("http://www.rumbledb.org/samples/git-archive-small.json")[1]

We can see that there are nested objects and arrays. This is perfect for JSONiq. Let us now figure out all the keys used in this dataset with the `keys()` function.

In [ ]:
%%jsoniq
keys(json-file("http://www.rumbledb.org/samples/git-archive-small.json"))

Let us look closer at the key called "type". What values does it take? We can use dot-based navigation to navigate down to these values. This will work nicely on the entire dataset.

In [ ]:
%%jsoniq
json-file("http://www.rumbledb.org/samples/git-archive-small.json").type

It looks like there are a lot of duplicates in there. Let us use `distinct-values()` to figure out all unique values. 

In [ ]:
%%jsoniq
distinct-values(json-file("http://www.rumbledb.org/samples/git-archive-small.json").type)

So we see that for the key "type", all values are strings and there are only... how many, by the way? Let us use count().

In [ ]:
%%jsoniq
count(distinct-values(json-file("http://www.rumbledb.org/samples/git-archive-small.json").type))

So there are 13. Note that `count()` works just as well on the entire dataset, to know how many objects there are.

In [ ]:
%%jsoniq
count(json-file("http://www.rumbledb.org/samples/git-archive-small.json"))

Let us know look at nested objects. It seems the key "actor" has these, so let us now use the dot object lookup to find all these values.

In [ ]:
%%jsoniq
json-file("http://www.rumbledb.org/samples/git-archive-small.json").actor

We can chain dot object lookups to navigate further down, for example to logins. Let us figure out how many distinct logins there are.

In [ ]:
%%jsoniq
count(distinct-values(json-file("http://www.rumbledb.org/samples/git-archive-small.json").actor.login))

The id field inside the actor object seems to be an integer. What is the highest value? The `max()` function also works at large scales, just like `count()` and also `min()`, `avg()` and `sum()`.

In [ ]:
%%jsoniq
max(json-file("http://www.rumbledb.org/samples/git-archive-small.json").actor.id)

Alright, let us know look for nested arrays. There does not seem to have any inside the actor object, so let us try the key "payload". Let us just look at the first one.

In [ ]:
%%jsoniq
json-file("http://www.rumbledb.org/samples/git-archive-small.json")[1].payload

Here we see that there is a nested array associated with key "commits".

In [ ]:
%%jsoniq
json-file("http://www.rumbledb.org/samples/git-archive-small.json")[1].payload.commits

In this case, there is only one object in this array. Is there, by any chance, any one of these arrays that has more than one commit? For this, we can use a Boolean predicate. Let us evaluate the predicate

`size($$) gt 1`

which uses the size function and the `gt` (greater than) comparison and where `$$` is the current array being tested.

In [ ]:
%%jsoniq
json-file("http://www.rumbledb.org/samples/git-archive-small.json").payload.commits[size($$) gt 1]

Let us just take the first one to have more visibility.

In [ ]:
%%jsoniq
json-file("http://www.rumbledb.org/samples/git-archive-small.json").payload.commits[size($$) gt 1][1]

We can expand it to a sequence of objects using the `[]` array unboxing syntax.

In [ ]:
%%jsoniq
json-file("http://www.rumbledb.org/samples/git-archive-small.json").payload.commits[size($$) gt 1][1][]

We can also lookup a specific position, say, the second object, with the `[[ ]]` array lookup syntax.

In [ ]:
%%jsoniq
json-file("http://www.rumbledb.org/samples/git-archive-small.json").payload.commits[size($$) gt 1][1][[2]]

And now, please hold for something awesome. We can unbox all arrays of the entire collection at the same time by just using the `[]` syntax on the entire dataset.

In [ ]:
%%jsoniq
json-file("http://www.rumbledb.org/samples/git-archive-small.json").payload.commits[]

These are objects. It is all too tempting to navigate further down with more dot object-lookup syntax. All at the same time, obviously. Let us figure out how many unique emails there are in all commits of all events.

In [ ]:
%%jsoniq
count(distinct-values(json-file("http://www.rumbledb.org/samples/git-archive-small.json").payload.commits[].author.email))

Now, how many unique emails are there in first commits?

In [ ]:
%%jsoniq
count(distinct-values(json-file("http://www.rumbledb.org/samples/git-archive-small.json").payload.commits[[1]].author.email))

You have now learned how to navigate large JSON datasets with the dot object lookup syntax, the `[]` array unboxing syntax, the `[[ ]]` array lookup syntax, number predicates, and Boolean predicates.

All of these work nicely on very large sequences, and you can chain them arbitrarily. In fact, this will all happen in parallel on the cores of your machine or even on a large cluster.

You also saw how to aggregate large sequences of values with min, max, count, avg and sum.

Finally, you saw how to eliminate duplicates with `distinct-values`.

# Variables

Some of the queries seen previously involve several chained lookups and function calls. It can become complex.

In [ ]:
%%jsoniq
count(distinct-values(json-file("http://www.rumbledb.org/samples/git-archive-small.json").actor.login))

It is then a natural thing to use variables to store intermediate results. This can be achieved with a series of let clauses. The final result is then put in a return clause.

In [ ]:
%%jsoniq
let $path := "http://www.rumbledb.org/samples/git-archive-small.json"
let $events := json-file($path)
let $actors := $events.actor
let $logins := $actors.login
let $distinct-logins := distinct-values($logins)
return count($distinct-logins)

Note that types are not needed, however they exist! It is possible to add a static type to each variable.
Since values can be sequences, you can add suffixes for cardinality: `*` for a sequence of arbitrary length, `?` for zero or one item, `+` for one or more items.

In [ ]:
%%jsoniq
let $path as string := "http://www.rumbledb.org/samples/git-archive-small.json"
let $events as object* := json-file($path)
let $actors as object* := $events.actor
let $logins as string* := $actors.login
let $distinct-logins as string* := distinct-values($logins)
let $count as integer := count($distinct-logins)
return $count

As you can see, variables can be used to store single items, as well as enormous sequences. RumbleDB will automatically select the best way to evaluate your query.

Note that it is possible to reuse variable names. However, these are not assignments: these are bindings. Reusing a variable name hides the previous binding.

In [ ]:
%%jsoniq
let $v as string := "http://www.rumbledb.org/samples/git-archive-small.json"
let $v as object* := json-file($v)
let $v as object* := $v.actor
let $v as string* := $v.login
let $v as string* := distinct-values($v)
let $v as integer := count($v)
return $v

## Iteration

It is possible to iterate on the elements in a sequence, like so:

In [ ]:
%%jsoniq
for $i in 1 to 10
return $i * 2

The sequence to iterator on can itself come from a dataset, such as the one we were using previously:

In [ ]:
%%jsoniq
for $event in json-file("http://www.rumbledb.org/samples/git-archive-small.json")
return size($event.payload.commits)

`for` clauses can be mixed with `let` clauses:

In [ ]:
%%jsoniq
let $path := "http://www.rumbledb.org/samples/git-archive-small.json"
for $event in json-file($path)
let $commits := $event.payload.commits
return size($commits)

And the results can also be nested in a more complex query: for example, let us compute the max of all these array sizes.

In [ ]:
%%jsoniq
max(
  let $path := "http://www.rumbledb.org/samples/git-archive-small.json"
  for $event in json-file($path)
  let $commits := $event.payload.commits
  return size($commits)
)

A third kind of clause is the `where` clause: it allows you to filter events. Let us only keep those with more than 10 commits, and count them.

In [ ]:
%%jsoniq
count(
  let $path := "http://www.rumbledb.org/samples/git-archive-small.json"
  for $event in json-file($path)
  let $commits := $event.payload.commits
  where size($commits) gt 10
  return $event
)

## Simple calculations

Let us now look closer arithmetics, comparison and logic expressions. They are particularly useful in a where clause or in a Boolean predicate, however these expressions can be used just about anywhere as this is a functional language.

### Arithmetics

JSONiq works like a calculator and can do arithmetics with the four basic operations.

In [ ]:
%%jsoniq
 (38 + 2) div 2 + 11 * 2


(mind the division operator which is the "div" keyword. The slash operator has different semantics).

Like JSON, JSONiq works with decimals and doubles:

In [ ]:
%%jsoniq
 6.022e23 * 42

JSONiq also support modulos, integer division, and has a rich function library (trigonometry, logarithms, exponential, powers, etc).

## Comparison

Values (numbers, strings, dates, etc) can be compared with the binary operators `eq`, `ne`, `gt`, `ge`, `lt` and `le`.
Let us change the comparison used in the where clause with other kinds.

In [ ]:
%%jsoniq
count(
  let $path := "http://www.rumbledb.org/samples/git-archive-small.json"
  for $event in json-file($path)
  let $commits := $event.payload.commits
  where size($commits) gt 10
  return $event
)

In [ ]:
%%jsoniq
count(
  let $path := "http://www.rumbledb.org/samples/git-archive-small.json"
  for $event in json-file($path)
  let $commits := $event.payload.commits
  where size($commits) eq 10
  return $event
)

In [ ]:
%%jsoniq
count(
  let $path := "http://www.rumbledb.org/samples/git-archive-small.json"
  for $event in json-file($path)
  let $commits := $event.payload.commits
  where size($commits) ne 10
  return $event
)

In [ ]:
%%jsoniq
count(
  let $path := "http://www.rumbledb.org/samples/git-archive-small.json"
  for $event in json-file($path)
  let $commits := $event.payload.commits
  where size($commits) le 10
  return $event
)

Why not = or < or >=? This is because these are more powerful. In fact, they implicitly perform an existential quantification over the operands.

In [ ]:
%%jsoniq
1 to 10 = 5

In [ ]:
%%jsoniq
1 to 10 > 11 to 20

### Logical operations

JSONiq supports Boolean logic.

In [ ]:
%%jsoniq
true and false

In [ ]:
%%jsoniq
(true or false) and (false or true)

The unary not is also available:

In [ ]:
%%jsoniq
not true

Note that JSONiq, unlike SQL, does two-valued logic. Nulls are automatically converted to false.

In [ ]:
%%jsoniq
null and true

Some non-Booleans can also get converted. For example, non-empty strings are converted to true and empty strings to false.

In [ ]:
%%jsoniq
not ""

In [ ]:
%%jsoniq
not "non empty"

Zero is converted to false, non-zero numbers to true.

In [ ]:
%%jsoniq
not 0

In [ ]:
%%jsoniq
not 1e10

### Strings

JSONiq is capable of manipulating strings as well, using functions:


In [ ]:
%%jsoniq
concat("Hello ", "Captain ", "Kirk")

In [ ]:
%%jsoniq
substring("Mister Spock", 8, 5)

JSONiq comes up with a rich string function library out of the box, inherited from its base language. These functions are listed [here](https://www.w3.org/TR/xpath-functions-30/) (actually, you will find many more for numbers, dates, etc).



### Sequences

Until now, we have only been working with single values (an object, an array, a number, a string, a boolean). JSONiq supports sequences of values. You can build a sequence using commas:


In [ ]:
%%jsoniq
 (1, 2, 3, 4, 5, 6, 7, 8, 9, 10)

In [ ]:
%%jsoniq
1, true, 4.2e1, "Life"

The "to" operator is very convenient, too:

In [ ]:
%%jsoniq
 (1 to 100)

Some functions even work on sequences:

In [ ]:
%%jsoniq
sum(1 to 100)

In [ ]:
%%jsoniq
string-join(("These", "are", "some", "words"), "-")

In [ ]:
%%jsoniq
count(10 to 20)

In [ ]:
%%jsoniq
avg(1 to 100)

Unlike arrays, sequences are flat. The sequence (3) is identical to the integer 3, and (1, (2, 3)) is identical to (1, 2, 3). See [this post](https://www.jsoniq.org/docs/Introduction_to_JSONiq/html/ch17.html#:~:text=Any%20value%20returned%20by%20or,them%20to%20form%20bigger%20sequences.) for more detail on sequences vs. arrays.

and even filter out some values:

In [ ]:
%%jsoniq
let $sequence := 1 to 10
for $value in $sequence
let $square := $value * $value
where $square < 10
return $square

Note that you can only iterate over sequences, not arrays. To iterate over an array, you can obtain the sequence of its values with the `[]` operator, like so:


In [ ]:
%%jsoniq
[1, 2, 3][]

Compare the following two queries:

In [ ]:
%%jsoniq
let $sequence := [1, 2, 3][]
for $value in $sequence
let $square := $value * $value
where $square < 10
return $square

In [ ]:
%%jsoniq
let $sequence := [1, 2, 3]
for $value in $sequence
let $square := $value * $value
where $square < 10
return $square

### Conditions

You can make the output depend on a condition with an `if-then-else` construct:

In [ ]:
%%jsoniq
for $x in 1 to 10
return if ($x < 5) then $x
                   else -$x

Note that the `else` clause is required - however, it can be the empty sequence `()` which is often when you need if only the `then` clause is relevant to you. For instance,

In [ ]:
%%jsoniq
for $x in 1 to 10
return if ($x < 5) then $x
                   else ()

### Composability of Expressions

Now that you know of a couple of elementary JSONiq expressions, you can combine them in more elaborate expressions. For example, you can put any sequence of values in an array:

In [ ]:
%%jsoniq
[ 1 to 10 ]

Or you can dynamically compute the value of object pairs (or their key):

In [ ]:
%%jsoniq
{
      "Greeting" : (let $d := "Mister Spock"
                    return concat("Hello, ", $d)),
      "Farewell" : string-join(("Live", "long", "and", "prosper"),
                               " ")
}

You can dynamically generate object singletons (with a single pair):


In [ ]:
%%jsoniq
{ concat("Integer ", 2) : 2 * 2 }

and then merge lots of them into a new object with the `{| |}` notation:

In [ ]:
%%jsoniq
{|
    for $i in 1 to 10
    return { concat("Square of ", $i) : $i * $i }
|}

## JSON Navigation

Up to now, you have learnt how to compose expressions so as to do some computations and to build objects and arrays. It also works the other way round: if you have some JSON data, you can access it and navigate.
All you need to know is: JSONiq views
an array as an ordered list of values,
an object as a set of key/value pairs.


### Objects

You can use the dot operator to retrieve the value associated with a key. Quotes are optional, except if the key has special characters such as spaces. It will return the value associated thereto:

In [ ]:
%%jsoniq
let $person := {
    "first name" : "Sarah",
    "age" : 13,
    "gender" : "female",
    "friends" : [ "Jim", "Mary", "Jennifer"]
}
return $person."first name"

You can also ask for all keys in an object:

In [ ]:
%%jsoniq
let $person := {
    "name" : "Sarah",
    "age" : 13,
    "gender" : "female",
    "friends" : [ "Jim", "Mary", "Jennifer"]
}
return { "keys" : [ keys($person)] }

### Arrays

The `[[]]` operator retrieves the entry at the given position:

In [ ]:
%%jsoniq
let $friends := [ "Jim", "Mary", "Jennifer"]
return $friends[[1+1]]

It is also possible to get the size of an array:

In [ ]:
%%jsoniq
let $person := {
    "name" : "Sarah",
    "age" : 13,
    "gender" : "female",
    "friends" : [ "Jim", "Mary", "Jennifer"]
}
return { "how many friends" : size($person.friends) }

Finally, the `[]` operator returns all elements in an array, as a sequence:

In [ ]:
%%jsoniq
let $person := {
    "name" : "Sarah",
    "age" : 13,
    "gender" : "female",
    "friends" : [ "Jim", "Mary", "Jennifer"]
}
return $person.friends[]

### Relational Algebra

Do you remember SQL's `SELECT FROM WHERE` statements? JSONiq inherits selection, projection and join capability from XQuery, too. Let's read the appetizer example we showed at the beginning of this tutorial again, at this point you should be able to understand it better. 

In [ ]:
%%jsoniq
let $stores :=
[
    { "store number" : 1, "state" : "MA" },
    { "store number" : 2, "state" : "MA" },
    { "store number" : 3, "state" : "CA" },
    { "store number" : 4, "state" : "CA" }
]
let $sales := [
    { "product" : "broiler", "store number" : 1, "quantity" : 20  },
    { "product" : "toaster", "store number" : 2, "quantity" : 100 },
    { "product" : "toaster", "store number" : 2, "quantity" : 50 },
    { "product" : "toaster", "store number" : 3, "quantity" : 50 },
    { "product" : "blender", "store number" : 3, "quantity" : 100 },
    { "product" : "blender", "store number" : 3, "quantity" : 150 },
    { "product" : "socks", "store number" : 1, "quantity" : 500 },
    { "product" : "socks", "store number" : 2, "quantity" : 10 },
    { "product" : "shirt", "store number" : 3, "quantity" : 10 }
]
let $join :=
    for $store in $stores[], $sale in $sales[]
    where $store."store number" = $sale."store number"
    return {
        "nb" : $store."store number",
        "state" : $store.state,
        "sold" : $sale.product
    }
return [$join]

### Access yet another dataset

RumbleDB can read input from many file systems and many file formats. If you are using our backend, you can only use `json-doc()` with any URI pointing to a JSON file and navigate it as you see fit. 

You can read data from your local disk, from S3, from HDFS, and also from the Web. For this tutorial, we'll read from the Web because, well, we are already on the Web.

We have put a sample at http://rumbledb.org/samples/products-small.json that contains 100,000 small objects like:



In [ ]:
%%jsoniq
json-file("http://rumbledb.org/samples/products-small.json", 10)[1]

In [ ]:
%%jsoniq
count(json-file("http://rumbledb.org/samples/products-small.json", 10))

The second parameter to json-file, 10, indicates to RumbleDB that it should organize the data in ten partitions after downloading it, and process it in parallel. If you were reading from HDFS or S3, the parallelization of these partitions would be pushed down to the distributed file system.

JSONiq supports the relational algebra. For example, you can do a selection with a where clause, like so:

In [ ]:
%%jsoniq
for $product in json-file("http://rumbledb.org/samples/products-small.json", 10)
where $product.quantity ge 995
return $product

Notice that by default only the first 200 items are shown. In a typical setup, it is possible to output the result of a query to a distributed system, so it is also possible to output all the results if needed. In this case, however, as this is printed on your screen, it is more convenient not to materialize the entire sequence.

For a projection, there is `project()`:

In [ ]:
%%jsoniq
for $product in json-file("http://rumbledb.org/samples/products-small.json", 10)
where $product.quantity ge 995
return project($product, ("store-number", "product"))

You can also page the results (like OFFSET and LIMIT in SQL) with a `count` clause and a `where` clause. The following query takes the 10th record to the 20th record in the dataset. 

In [ ]:
%%jsoniq
for $product in json-file("http://rumbledb.org/samples/products-small.json", 10)
where $product.quantity ge 995
count $c
where $c gt 10 and $c le 20
return project($product, ("store-number", "product"))

JSONiq also supports grouping with a `group by` clause:

In [ ]:
%%jsoniq
for $product in json-file("http://rumbledb.org/samples/products-small.json", 10)
group by $store-number := $product.store-number
return {
    "store" : $store-number,
    "count" : count($product)
}

As well as ordering with an `order by` clause:

In [ ]:
%%jsoniq
for $product in json-file("http://rumbledb.org/samples/products-small.json", 10)
group by $store-number := $product.store-number
order by $store-number ascending
return {
    "store" : $store-number,
    "count" : count($product)
}

JSONiq supports denormalized data, so you are not forced to aggregate after a grouping, you can also nest data like so:

In [ ]:
%%jsoniq
for $product in json-file("http://rumbledb.org/samples/products-small.json", 10)
group by $store-number := $product.store-number
order by $store-number ascending
return {
    "store" : $store-number,
    "products" : [ distinct-values($product.product) ]
}

Or

In [ ]:
%%jsoniq
for $product in json-file("http://rumbledb.org/samples/products-small.json", 10)
group by $store-number := $product.store-number
order by $store-number ascending
return {
    "store" : $store-number,
    "products" : [ project($product[position() le 10], ("product", "quantity")) ],
    "inventory" : sum($product.quantity)
}

See the explanation of predict on positions `position()` [here](https://rumble.readthedocs.io/en/latest/JSONiq/). 

That's it! You know the basics of JSONiq. 

<!-- Now you can also download the RumbleDB jar and run it on your own laptop. Or [on a Spark cluster, reading data from and to HDFS](https://rumble.readthedocs.io/en/latest/Run%20on%20a%20cluster/), etc. -->




Feel free to try the many functions available, here is a link with the exhaustive list: https://rumble.readthedocs.io/en/latest/Function%20library/. 


# Moodle quiz (11.2): querying a bigger git-archive dataset

You will have to submit the results of this exercise to Moodle to obtain the weekly bonus. You will need these things:
- The query you wrote (ungraded)
- Something related to its output (which you will be graded on)
- The time it took you to run it (ungraded)

The execution time of the queries will be reported by Rumble.

### Check the data
We provide you with a bigger git-archive dataset [git-archive-big.json](https://www.rumbledb.org/samples/git-archive-big.json), you can already check that you get the correct number of records. The dataset should contain 206978 records. You can either use `wget` to download and read the dataset locally or simply read with `json-file` from the URI.

In [ ]:
%%jsoniq
count(for $i in json-file("git-archive-big.json")
return $i)

In [ ]:
%%jsoniq
count(for $i in json-file("https://www.rumbledb.org/samples/git-archive-big.json")
return $i)

### Question 1: Which actor commited the most events of the type "PushEvent", and how many did they commit? 

Answer with the actor id, and the frequency of its push events. 

### Question 2: Who is the actor that has committed the largest set of distinct event types? 

Answer with the actor id.

Hint: a priori there could be more than one, but in fact there is only one with the highest number of distinct event types.

### Question 3: At what time was the most recent fork of the repository "bitcoin/bitcoin" created?


Important: Provide the hour, minute and seconds of the most recent one as numbers (e.g. if the timestamp is "2018-01-01T14:20:59Z" provide only 14 20 59)


# 4. Creating Parquet documents using RumbleDB and JSound (optional)
In this exercise, we will be writing a JSONiq query which validates a document using a JSound schema to create a user defined type, which we can then query! Then, we will output it in the [Parquet](https://parquet.apache.org/) file format. 

### 4.1 JSound Schema for the Great Language Game
The [Great Language Game](http://greatlanguagegame.com/) is a game in which you are given a voice clip to listen, and you are asked to identify the language in which the person was speaking. It is a multiple-choice question–you make your choice out of several alternatives.

The following JSON document presents a user's attempt at answering a single question in the game: it contains the identifier of the voice clip, the choices presented to the player, and the player's response. 

As we see in [exercise06](https://github.com/RumbleDB/bigdata-exercises/blob/47dc2098dd494f25155e449639c41c1bc1d7fd92/Big_Data/exercise06/Exercise06_DataModels_Solution.ipynb), [JSound](http://www.jsound-spec.org/) is a vocabulary that allows you to validate JSON documents. It employs a very simple and intuitive JSON-like syntax.

Your task: provide a JSound Schema which will validate this document. 
```json
{
  "guess": "Norwegian",
  "target": "Norwegian", 
  "country": "AU",
  "choices": [ "Maori", "Mandarin", "Norwegian", "Tongan" ], 
  "sample": "48f9c924e0d98c959d8a6f1862b3ce9a",
  "date": "2013-08-19"
}
```

You can refer to the documentation for [a list of types in JSound](http://www.jsound-spec.org/publish/en-US/JSound/2.0/html-single/JSound/index.html#idm126).


### Solution

The following JSound schema is a possible solution to the above question:

```json
{
    "guess": "string",
    "target": "string",
    "country": "string",
    "choices": [ "string" ],
    "sample": "hexBinary",
    "date": "date"
}
```

### 4.2 Validating our first documents
Now let's validate the above document with the schema we just wrote:

In [ ]:
%%jsoniq
declare type local:attempt as {
    // Replace this object with your JSound schema!
    // Remember to keep the semi-colon :)
};

validate type local:attempt* {
  {
    "guess": "Norwegian",
    "target": "Norwegian", 
    "country": "AU",
    "choices": [ "Maori", "Mandarin", "Norwegian", "Tongan" ], 
    "sample": "48f9c924e0d98c959d8a6f1862b3ce9a",
    "date": "2013-08-19"
  }
}

Let's work with a larger version of the Great Language Game (Confusion) dataset, stored in the JSON format. The file should already be in the local directory as `confusion-100000.json`.

We can query this file directly on the local disk using Rumble:

In [ ]:
%%jsoniq
count(json-file("confusion-100000.json"))

Now, let's use our schema to validate the first 10 attempts from the dataset:

In [ ]:
%%jsoniq
declare type local:attempt as {
    // Replace this object with your JSound schema!
    // Remember to keep the semi-colon :)
};

validate type local:attempt* {
    json-file("confusion-100000.json")[position() <= 10]
}

Although it seems like nothing is happening, our query is returning _typed_ objects instead of JSON strings. This means that we can interact with values like dates in more meaningful ways:

In [ ]:
%%jsoniq
declare type local:attempt as {
    // Replace this object with your JSound schema!
    // Remember to keep the semi-colon :)
};

for $i in validate type local:attempt* {
    json-file("confusion-100000.json")[position() <= 10]
}
let $date := $i.date
return month-from-date($date)

### Solution

In [ ]:
%%jsoniq
declare type local:attempt as {
    "guess": "string",
    "target": "string",
    "country": "string",
    "choices": [ "string" ],
    "sample": "hexBinary",
    "date": "date"
};

validate type local:attempt* {
  {
    "guess": "Norwegian",
    "target": "Norwegian", 
    "country": "AU",
    "choices": [ "Maori", "Mandarin", "Norwegian", "Tongan" ], 
    "sample": "48f9c924e0d98c959d8a6f1862b3ce9a",
    "date": "2013-08-19"
  }
}

In [ ]:
%%jsoniq
declare type local:attempt as {
    "guess": "string",
    "target": "string",
    "country": "string",
    "choices": [ "string" ],
    "sample": "hexBinary",
    "date": "date"
};

validate type local:attempt* {
    json-file("confusion-100000.json", 10)[position() <= 10]
}

In [ ]:
%%jsoniq
declare type local:attempt as {
    "guess": "string",
    "target": "string",
    "country": "string",
    "choices": [ "string" ],
    "sample": "hexBinary",
    "date": "date"
};

for $i in validate type local:attempt* {
  json-file("confusion-100000.json")[position() <= 10]
}
let $date := $i.date
return month-from-date($date)

### 4.3 From JSON to Parquet
Now, let's try validate our entire dataset and output it as a Parquet file! We'll need the shell for this, so let's first create a JSONiq file containing our query. Modify and then copy the following query into a new file called `query.jq`.

**Hint:** Date types in JSound can have timezones, but Parquet does not support dates with timezones at the moment. We should instead validate dates as strings for this exercise.

```json
declare type local:attempt as {
    // Replace this object with your JSound schema!
    // Remember to keep the semi-colon :)
};

validate type local:attempt* {
    json-file("confusion-100000.json", 10)
}
```

Then, we can run the query through the shell, specifying the output format as Parquet.

Run the following tests in a shell under the exercise directory of this week, or simply here

In [ ]:
!java -jar rumbledb-1.20.0-standalone.jar run -q '1+1'

In [ ]:
!java -jar rumbledb-1.20.0-standalone.jar run query.jq -o result.out -f parquet -P 1

Then let's change the file name to be more representative of its contents:

In [ ]:
!cp `find result.out/part-00000*` greatlanguagegame.parquet

Where, if we compare the sizes of the JSONL and Parquet files...

In [ ]:
!ls -lh

...we can see how much smaller the Parquet file is! Parquet is a column-oriented binary storage format with efficient data compression schemes.

Intuitively, reducing the size of a file while maintaining fixed throughput means that we can also scan the file much faster! Keep this in mind next time you need to work with a huge JSON dataset!

In [ ]:
%%jsoniq
count(parquet-file("greatlanguagegame.parquet"))

### 4.4 Bigger data for a bigger benefit 

The dataset we used was just a subset of the full great language game dataset. When using a larger dataset, we will observe a much more noticeable decrease in time to scan the dataset in Parquet over JSON!

We can download this dataset here:

In [ ]:
!wget -O- http://data.greatlanguagegame.com.s3.amazonaws.com/confusion-2014-03-02.tbz2 | tar -jxv

Then, let's try repeat several steps from above with the new dataset:

In [ ]:
%%jsoniq
count(json-file("confusion-2014-03-02/confusion-2014-03-02.json"))

Including creating a new query file to point to the larger dataset:

```json
declare type local:my-type as {
    // Replace this object with your JSound schema!
    // Remember to keep the semi-colon :)
};

validate type local:my-type* {
    json-file("confusion-2014-03-02/confusion-2014-03-02.json", 10)
}
```

which should be named `query-large.jq`.

In [ ]:
!java -jar rumbledb-1.20.0-standalone.jar run query-large.jq -o result-large.out -f parquet -P 1

In [ ]:
!cp `find result-large.out/part-00000*` greatlanguagegame-large.parquet

In [ ]:
%%jsoniq
count(parquet-file("greatlanguagegame-large.parquet"))